# Challenge MELI

## Importe de librerias y funciones necesarias para el challenge

En general se necesitan las librerias de Drive API (**google** y **googleapiclient**), para hacer la autenticacion oauth; **pandas**, para el manejo y filtrado de datos; **sqlite**, para guardar la BD y **smtplib** para el envio de correos.

In [100]:
import os
import os.path
import io
from urllib.error import HTTPError
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd
import sqlite3 as db
import smtplib, ssl
from email.message import EmailMessage
from config import usr,pwd

In [108]:
def send_email(rcpt,nfile):
    # Create a secure SSL context
    context = ssl.create_default_context()
    msg = EmailMessage()
    msg['Subject'] = 'Challenge MELI | {}'.format(nfile)
    msg['From'] = usr
    msg['To'] = rcpt
    message = """
    Cambiando la privacidad para el archivo {} a privado.
    """.format(nfile)
    msg.set_content(message)
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(usr, pwd)
        server.send_message(msg)
        
def delete_permisssions(row):
    service.permissions().delete(fileId=row['id'],permissionId="anyoneWithLink").execute()
    send_email(row['emailAddress'],row['nombre'])
    return True

## Establecer conexiones

### Creacion y/o conexion a la BD Sqlite

In [2]:
connection = db.connect('meli.db')

In [101]:
try:
    old_df = pd.read_sql_query("SELECT * from filespermissions", connection)   
         
except db.OperationalError:
    print("No such table: filespermissions")
    if(db.OperationalError):
        try:
            print("Creating a new table: ")
            cur = connection.cursor()
            cur.execute('''
                CREATE TABLE filespermissions(
                emailAddress text, 
                id text,
                nombre text,
                extension text,
                owner text,
                compartido	integer
                propietario	integer
                publico	text
                modifiedTime text,
                Modificado	integer
            )''')
            cur.close()
 
        except db.Error() as e:
            print(e, " occured")

 

### Obtener autorizacion para acceder a los archivos del Drive de Google

<img src="https://developers.google.com/static/identity/protocols/oauth2/images/flows/authorization-code.png" 
        alt="Flujo-oauth" 
        width="800" 
        height="500" 
        style="display: block; margin: 0 auto" />

In [102]:
SCOPES = ['https://www.googleapis.com/auth/drive']

In [103]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json',SCOPES)
    
if not creds or not creds.valid:
    
    if creds and creds.expired and creds.refresh_token:
        
        creds.refresh(Request())
        
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=8080)
        
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

## Creacion de BD sobre el estado actual de los archivos

In [104]:
lists = []
try:
    
    service =  build('drive', 'v3' , credentials = creds)
    ##### OPERACION DE LISTADO DE ARCHIVOS #############
    #results =  service.files().list(fields = "*",spaces='drive').execute()
    results =  service.files().list(q="trashed = false and 'me' in owners and mimeType != 'application/vnd.google-apps.folder'",
                                          spaces='drive',
                                          pageSize=1000,
                                          fields='nextPageToken, files(id,name,shared,ownedByMe,fullFileExtension,owners,permissions,modifiedTime)').execute()
    items = results.get('files',[])
    if not items:
        print("No hay items")
    for item in items:
        try:
            #print(item['name'],item['fullFileExtension'],item['owners'][0]['displayName'],item['shared'],item['ownedByMe'])
            lists.append([item['owners'][0]['emailAddress'],item['id'],item['name'],item['fullFileExtension'],item['owners'][0]['displayName'],item['shared'],item['ownedByMe'],item['permissions'][0]['id'],item['modifiedTime']])
            #print(lists)
        except:
            pass
            #print(item['name'],item['name'].split('.')[-1],item['owners'][0]['displayName'],item['shared'],item['ownedByMe'])
    

except HttpError as error:
    print(f'Ocurrió un error {error}')

In [105]:
print(len(lists))
while True:
    if 'nextPageToken' in results:
        nextToken = results["nextPageToken"]
        results =  service.files().list(q="trashed = false and 'me' in owners and mimeType != 'application/vnd.google-apps.folder'",
                                            spaces='drive',
                                            pageSize=1000,
                                            fields='nextPageToken, files(id,name,shared,ownedByMe,fullFileExtension,owners,permissions,modifiedTime)',
                                            pageToken=nextToken).execute()
        items = results.get('files',[])
        if not items:
            print("No hay items")
        for item in items:
            try:
                #print(item['name'])
                #print(item['name'],item['fullFileExtension'],item['owners'][0]['displayName'],item['shared'],item['ownedByMe'])
                #print([item['owners'][0]['emailAddress'],item['id'],item['name'],item['fullFileExtension'],item['owners'][0]['displayName'],item['shared'],item['ownedByMe'],item['permissions'][0]['id'],item['modifiedTime']])
                lists.append([item['owners'][0]['emailAddress'],item['id'],item['name'],item['fullFileExtension'],item['owners'][0]['displayName'],item['shared'],item['ownedByMe'],item['permissions'][0]['id'],item['modifiedTime']])
            except:
                pass
                #print(item['name'],item['name'].split('.')[-1],item['owners'][0]['displayName'],item['shared'],item['ownedByMe'])
        
    else:
        print('Ya no hay mas archivos')
        break
print(len(lists))

87
Ya no hay mas archivos
957


In [106]:
df = pd.DataFrame(lists, columns=['emailAddress','id','nombre','extension','owner','compartido','propietario','publico','modifiedTime'])
df['Modificado'] = False

In [109]:
df.head()

,emailAddress,id,nombre,extension,owner,compartido,propietario,publico,modifiedTime,Modificado
0,yampi16@gmail.com,1uzDYLf5QAPEFbr7oCq6ETnttKGb5sWwK,1517020241702.jfif,jfif,jean pierre,True,True,anyoneWithLink,2022-11-06T18:16:02.101Z,False
1,yampi16@gmail.com,1nmtIpznBILhm9lvWeEDPIzcnrKrqlFyU,PGPMessage.pdf,pdf,jean pierre,True,True,anyoneWithLink,2022-11-06T17:34:35.796Z,False
2,yampi16@gmail.com,1z11Krpq7T-iKJ39SoizWHG33BegEdBjQ,Compilado.rar,rar,jean pierre,False,True,03709418903600911746,2021-10-21T02:19:37.000Z,False
3,yampi16@gmail.com,1qP63HpQBizimaSOwINdzjO_2bJTxcR-o,DeRoze.rar,rar,jean pierre,False,True,03709418903600911746,2021-10-12T02:11:57.000Z,False
4,yampi16@gmail.com,1OJpO4g7oebdd2s0l-vDhOUFQUtjnBKrY,Presentacion.mp4,mp4,jean pierre,False,True,03709418903600911746,2021-10-12T02:10:22.000Z,False


In [110]:
old_df.head()

,emailAddress,id,nombre,extension,owner,compartido,propietario,publico,modifiedTime,Modificado
0,yampi16@gmail.com,1uzDYLf5QAPEFbr7oCq6ETnttKGb5sWwK,1517020241702.jfif,jfif,jean pierre,0,1,03709418903600911746,2022-11-05T14:57:47.850Z,1
1,yampi16@gmail.com,1z11Krpq7T-iKJ39SoizWHG33BegEdBjQ,Compilado.rar,rar,jean pierre,0,1,03709418903600911746,2021-10-21T02:19:37.000Z,0
2,yampi16@gmail.com,1qP63HpQBizimaSOwINdzjO_2bJTxcR-o,DeRoze.rar,rar,jean pierre,0,1,03709418903600911746,2021-10-12T02:11:57.000Z,0
3,yampi16@gmail.com,1OJpO4g7oebdd2s0l-vDhOUFQUtjnBKrY,Presentacion.mp4,mp4,jean pierre,0,1,03709418903600911746,2021-10-12T02:10:22.000Z,0
4,yampi16@gmail.com,1UYe-AHn2TJuXU821JAFJo9Jln4unw89D,soat.pdf,pdf,jean pierre,0,1,03709418903600911746,2021-09-21T11:30:59.539Z,0


## Actualizacion de permisos

En la siguiente linea se cambian los permisos y se notifica al usuario de ello

In [112]:
df['Modificado'] = df.apply(lambda x: delete_permisssions(x) if x['publico']=='anyoneWithLink' else x['Modificado'], axis=1)

<img src="https://i.imgur.com/vxXOTqh.png" 
        alt="correos" 
        width="1000" 
        height="200" 
        style="display: block; margin: 0 auto" />

<img src="https://i.imgur.com/gOvzzxd.png" 
        alt="contenido" 
        width="1000" 
        height="200" 
        style="display: block; margin: 0 auto" />

## Actualizacion de la base de datos

A continuacion se listan los archivos nuevos que se han encontrado desde la ultima corrida:

In [117]:
df[df['id'].isin(set(df['id'])-set(old_df['id']))]

,emailAddress,id,nombre,extension,owner,compartido,propietario,publico,modifiedTime,Modificado


In [113]:
diff_df = df[df['id'].isin(set(df['id'])-set(old_df['id']))]
print('Hay: ',len(diff_df),' archivos nuevos',end=' ')
if len(diff_df)>0:
    old_df = pd.concat([old_df, df], ignore_index = True)
else:
    print('no hay archivos nuevos desde la ultima corrida')

956
1


Se guarda los valores en la BD:

In [115]:
old_df.update(df)

In [116]:
old_df.head()

,emailAddress,id,nombre,extension,owner,compartido,propietario,publico,modifiedTime,Modificado
0,yampi16@gmail.com,1uzDYLf5QAPEFbr7oCq6ETnttKGb5sWwK,1517020241702.jfif,jfif,jean pierre,True,True,anyoneWithLink,2022-11-06T18:16:02.101Z,True
1,yampi16@gmail.com,1nmtIpznBILhm9lvWeEDPIzcnrKrqlFyU,PGPMessage.pdf,pdf,jean pierre,True,True,anyoneWithLink,2022-11-06T17:34:35.796Z,True
2,yampi16@gmail.com,1z11Krpq7T-iKJ39SoizWHG33BegEdBjQ,Compilado.rar,rar,jean pierre,False,True,03709418903600911746,2021-10-21T02:19:37.000Z,False
3,yampi16@gmail.com,1qP63HpQBizimaSOwINdzjO_2bJTxcR-o,DeRoze.rar,rar,jean pierre,False,True,03709418903600911746,2021-10-12T02:11:57.000Z,False
4,yampi16@gmail.com,1OJpO4g7oebdd2s0l-vDhOUFQUtjnBKrY,Presentacion.mp4,mp4,jean pierre,False,True,03709418903600911746,2021-10-12T02:10:22.000Z,False


In [15]:
old_df.to_sql("filespermissions", connection, if_exists="replace",index=False)


956

In [ ]:
connection.commit()
connection.close()